In [3]:
import sys
from FDMBuilder.FDMTable import *
from FDMBuilder.FDMDataset import *
from FDMBuilder.testing_helpers import *

## TO DO:

* PRIORITY: Work our what happens if you re-run .build() on a dataset
* re-running .build() needs "outside_obs" tables to be added back to src tables - TEST THIS


* THEN: build FDM datasets from just dataset_id:
    + summarise tables in dataset init i.e. which are ready for FDM build, which arent, any FDM tables that will be overwritten, any src tables that will be re-combined
    * think about "verbose" functionality of FDMTables - silence init-messages (not required anyways as tables will be initialised from tables in FDM dataset) - "check_build" function
    
    
* add check and warning for Null datetimes when adding start/end dates
* add .drop() method to table
* raw input functionality - copying src tables? overwriting existing tables?
* build data dict function
* ask about requirement to have a raw version of FDM_tables at all times? - read only source?

## Testing:

In [ ]:
build_test_environment()

In [4]:
table_1 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_1",
    dataset_id="CY_TESTS_FDM"
)

In [5]:
table_1.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_1 #####
________________________________________________________________________________

1. Copying src_table_1 to CY_TESTS_FDM:

    Table src_table_1 copied to CY_TESTS_FDM!

2. Adding person_id column:
    src_table_1 already contains person_id column

3. Adding event_start_date column:

    An event start date is required to build the observation_period table. This 
    information should be contained within one or more columns of your table. 
    If unsure a quick look at the table data in BigQuery should clarify.
    
    To start, is the event start data found in one column that can be easily 
    parsed with a day, month and year?
    > Type y or n y

    Which column contains the event start date?
    > Type the name (case sensitive): start_date

    What format does the date appear in YMD/YDM/DMY/MDY?
    > Type one: YMD
    event_start_date column added

4. Adding event_end_date column:

    An event end date may or may no

In [ ]:
table_1.quick_build(event_start_date_cols="start_date",
                     event_start_date_format="YMD",
                     event_end_date_cols="end_date",
                     event_end_date_format="YMD")

In [6]:
table_2 = FDMTable(
    source_table_full_id = f"{PROJECT}.CY_TESTS_SRC.src_table_2",
    dataset_id = "CY_TESTS_FDM" 
)

In [8]:
table_2.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_2 #####
________________________________________________________________________________

1. Copying src_table_2 to CY_TESTS_FDM:

    A copy of src_table_2 already exists in CY_TESTS_FDM. 
    You can continue with the existing src_table_2 table in CY_TESTS_FDM
    or make a fresh copy from the source dataset."   
    
    Continue with existing copy?
    > Type y or n: n

    Table src_table_2 copied to CY_TESTS_FDM!

2. Adding person_id column:
    person_id column added

3. Adding event_start_date column:

    An event start date is required to build the observation_period table. This 
    information should be contained within one or more columns of your table. 
    If unsure a quick look at the table data in BigQuery should clarify.
    
    To start, is the event start data found in one column that can be easily 
    parsed with a day, month and year?
    > Type y or n n

    We'll build the event start date beginning with the 

In [ ]:
table_2.quick_build(
    event_start_date_cols=["start_year", "start_month", "start_day"], 
    event_start_date_format="YMD", 
    event_end_date_cols=["end_year", "end_month", "end_day"], 
    event_end_date_format="YMD"
)

In [ ]:
table_3 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_3",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_3.quick_build(
    event_start_date_cols=["year", "month", "15"], 
    event_start_date_format="YMD", 
)

In [9]:
table_4 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_4",
    dataset_id="CY_TESTS_FDM"
)

In [10]:
table_4.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_4 #####
________________________________________________________________________________

1. Copying src_table_4 to CY_TESTS_FDM:

    Table src_table_4 copied to CY_TESTS_FDM!

2. Adding person_id column:
    src_table_4 already contains person_id column

3. Adding event_start_date column:

    An event start date is required to build the observation_period table. This 
    information should be contained within one or more columns of your table. 
    If unsure a quick look at the table data in BigQuery should clarify.
    
    To start, is the event start data found in one column that can be easily 
    parsed with a day, month and year?
    > Type y or n y

    Which column contains the event start date?
    > Type the name (case sensitive): start_date_string

    What format does the date appear in YMD/YDM/DMY/MDY?
    > Type one: DMY

    70 will be parsed as 2070. Consider converting year.
                
    event_start_date c

In [ ]:
table_4.quick_build(
    event_start_date_cols="start_date_string",
    event_start_date_format="MDY", 
    event_end_date_cols="end_date_string",
    event_end_date_format="DMY"
)

In [11]:
table_5 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_5",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_5.copy_table_to_dataset()

In [16]:
table_5.build()

	 ##### BUILDING FDM TABLE COMPONENTS FOR src_table_5 #####
________________________________________________________________________________

1. Copying src_table_5 to CY_TESTS_FDM:

    A copy of src_table_5 already exists in CY_TESTS_FDM. 
    You can continue with the existing src_table_5 table in CY_TESTS_FDM
    or make a fresh copy from the source dataset."   
    
    Continue with existing copy?
    > Type y or n: y

    Continuing with existing copy of src_table_5

2. Adding person_id column:
    src_table_5 already contains person_id column

3. Adding event_start_date column:

    An event start date is required to build the observation_period table. This 
    information should be contained within one or more columns of your table. 
    If unsure a quick look at the table data in BigQuery should clarify.
    
    To start, is the event start data found in one column that can be easily 
    parsed with a day, month and year?
    > Type y or n n

    We'll build the event star

In [14]:
table_5.add_column("SPLIT(date_string, '-')[OFFSET(1)] AS year")

In [15]:
table_5.add_column("SPLIT(date_string, '-')[OFFSET(0)] AS month")

In [ ]:
table_5.add_column("SPLIT(date_string, '-')[OFFSET(0)] AS year")
table_5.add_column("SPLIT(date_string, '-')[OFFSET(1)] AS month")
table_5.head()

In [ ]:
table_5.quick_build(
    event_start_date_cols=["year", "month", "15"],
    event_start_date_format="YMD", 
)

In [ ]:
table_6 = FDMTable(
    source_table_full_id=f"{PROJECT}.CY_TESTS_SRC.src_table_6",
    dataset_id="CY_TESTS_FDM"
)

In [ ]:
table_6.copy_table_to_dataset()

In [ ]:
table_6.head()

In [ ]:
table_6.quick_build(
    event_start_date_cols="date_string",
    event_start_date_format="DMY",
)

In [17]:
dataset = FDMDataset(
    dataset_id = "CY_TESTS_FDM"
)

In [18]:
dataset.build()

		 ##### BUILDING FDM DATASET CY_TESTS_FDM #####
________________________________________________________________________________

Checking dataset for source tables:

    src_table_1 contains:  * person_id * event_start_date  * event_end_date
	-> Table ready
    src_table_2 contains:  * person_id * event_start_date 
	-> Table ready
    src_table_4 contains:  * person_id * event_start_date 
	-> Table ready
    src_table_5 contains:  * person_id * event_start_date 
	-> Table ready

Building person table

    * Person table built with 60 entries

Building initial observation_period table

    * observation_period table built with 75 entries

4. Separating out problem entries from source tables

    src_table_1:
	* 19 problem entries identified and removed to src_table_1_problems
	* 6 entries remain in src_table_1
    src_table_2:
	* 20 problem entries identified and removed to src_table_2_problems
	* 5 entries remain in src_table_2
    src_table_4:
	* 17 problem entries identified and re

In [ ]:
blah = "got_money_mo_problems"
blah[:-9]